In [1]:
!pip install sentencepiece
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 93.9 MB/s eta 0:00:00


In [ ]:
## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

In [ ]:
!wget https://www.openslr.org/resources/12/train-clean-100.tar.gz

In [ ]:
!wget https://www.openslr.org/resources/12/test-clean.tar.gz

In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz

In [ ]:
!tar xvzf train-clean-100.tar.gz

In [ ]:
!tar xvzf dev-clean.tar.gz

In [ ]:
!tar xvzf test-clean.tar.gz

In [4]:
import torch
from torch import nn
import numpy as np
from torchaudio.models import RNNT,Emformer,emformer_rnnt_base,emformer_rnnt_model
from torch import optim
import torch
from torch import nn
from torchaudio import functional as audioF
from torchaudio import transforms as audioT
import torchaudio
import Levenshtein as Lev

In [5]:
import numpy as np
import torch
from torch import nn
from glob import glob
#import librosa
from tqdm import tqdm
#import soundfile as sf
import os
import random
from pathlib import Path

In [6]:
def wer(s1,s2):
    b = set(s1.split()+s2.split())
    word_map = dict(zip(b,range(len(b))))
    print("word_map:",word_map)
    w1 = [chr(word_map[w]) for w in s1.split()]
    w2 = [chr(word_map[w]) for w in s2.split()]
    dev = max(len(s2.split()),1)
    score = Lev.distance("".join(w1),"".join(w2))/dev
    return score

def cer(s1,s2):
    w1 = s1.replace(" ","")
    w2 = s2.replace(" ","")
    dev = max(len(s2),1)
    score = Lev.distance((w1),(w2))/dev
    return score

In [12]:
class DataLoader:
    def __init__(self,batch_size=1,is_test_set=False):
        super().__init__()
        self.batch_size = batch_size
        self.is_test_set = is_test_set
        #common_folder = "LibriSpeech/dev-clean"#
        common_folder = "LibriSpeech/train-clean-100"
        
        if(is_test_set==True):
            common_folder = "LibriSpeech/test-clean"
        transcripts_files = glob(os.path.join(common_folder,"*\\*\\*.txt"),recursive=True)
        audio_files = glob(os.path.join(common_folder,"*\\*\\*.flac"),recursive=True)
        if(is_test_set==False):
            audio_files_len = len(audio_files)
            audio_files = audio_files[:audio_files_len//1]
        print("transcripts_files:",len(transcripts_files))
        print("audio_files:",len(audio_files))
        transcripts = {}
        for ts_fpath in transcripts_files:
            with open(ts_fpath,"r") as fp:
                txt = fp.read().split("\n")
                for ln in txt:
                    ln=ln.strip()
                    if(ln==""):
                        continue
                  
                    lines = ln.split(" ",1)
                    fn = lines[0]
                    lbls = lines[1].upper()
                    transcripts[fn] = lbls
        self.audio_files = audio_files
        self.transcript_keys = list(transcripts.keys())
        self.transcript_table = "".join([chr(ord("A")+i) for i in range(26)])+(" .'!?")
        #print("transcript",transcripts)
        #print("transcript keys:",self.transcript_keys)
        
        self.transcripts = transcripts
        self._audio_file_len = len(self.audio_files)
        if(is_test_set==False):
            self.shuffleDataset()

    def __len__(self):
        return len(self.audio_files)//self.batch_size
    def _loadFeature(self,fpath):
        rawinput,sample_rate = torchaudio.load(fpath) #sf.read(fpath)
        audio_transform = audioT.MFCC(sample_rate=sample_rate,n_mfcc=96,log_mels=True)
        mfcc = audio_transform(rawinput)
        mfcc = torch.nn.functional.pad(mfcc,pad=(0,4)) # emformer rnn-t 가 이전 문맥과 자연스럽게 연결짓기 위해서 right context padding 을 가짐
        mfcc = mfcc.squeeze(dim=0).transpose(1,0)
        return mfcc
    def _convertTranscript2(self,text_list,with_blank=False):
        result = []
        for char in text_list:
            result.append(self.transcript_table.find(char)+3)
            if(with_blank==True):
                result.append(2)
        return result
    def shuffleDataset(self):
        random.shuffle(self.audio_files)
    def token2String(self,token_list,with_visible_tag=True):
        result = ""
        max_len = len(self.transcript_table)+2
        for i in range(len(token_list)):
            token_id = token_list[i]
            if(token_id==1):
                result+='<s>' if(with_visible_tag==True) else ""
            elif(token_id==0):
                result += '</s>' if(with_visible_tag==True) else ""
            elif(token_id==2):
                result += '</b>' if(with_visible_tag==True) else ""
            elif(2<token_id<max_len):
                result+= self.transcript_table[token_id-3]
            else:
                result+='</u?>' if(with_visible_tag==True) else ""
        return result
    
    def __getitem__(self,idx):
        if(idx>=self.__len__()):
            raise StopIteration()
        start_index,end_cond = (idx*self.batch_size,min((idx+1)*self.batch_size,(self._audio_file_len)))
        #print((start_index,end_cond))
        specs = []
        transcripts = []
        transcript_lengths = []
        specs_lengths = []

        for i in range(start_index,end_cond):
            fpath = self.audio_files[i]
            feature = self._loadFeature(fpath)
            fn = Path(fpath).stem
            specs.append(feature)#torch.from_numpy(feature))
            specs_lengths.append(feature.size(0))
            transcript_indices = [0]+self._convertTranscript2(self.transcripts[fn],with_blank=False)#+[2]
            transcripts.append(torch.from_numpy(np.array(transcript_indices)))
            transcript_lengths.append(len(transcript_indices))
        transcript_lengths = np.array(transcript_lengths)
        specs_lengths = np.array(specs_lengths)
        return specs,specs_lengths,transcripts,transcript_lengths

In [1]:

chosung = ("ㄱ", "ㄲ", "ㄴ", "ㄷ", "ㄸ", "ㄹ", "ㅁ", "ㅂ", "ㅃ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅉ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

jungsung = ("ㅏ", "ㅐ", "ㅑ", "ㅒ", "ㅓ", "ㅔ", "ㅕ", "ㅖ", "ㅗ", "ㅘ", "ㅙ", "ㅚ", "ㅛ", "ㅜ", "ㅝ", "ㅞ", "ㅟ", "ㅠ", "ㅡ", "ㅢ", "ㅣ")

jongsung = ("", "ㄱ", "ㄲ", "ㄳ", "ㄴ", "ㄵ", "ㄶ", "ㄷ", "ㄹ", "ㄺ", "ㄻ", "ㄼ", "ㄽ", "ㄾ", "ㄿ", "ㅀ", "ㅁ", "ㅂ", "ㅄ", "ㅅ", "ㅆ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ")

ENGS = tuple([chr(ord("A")+i) for i in range(26)])

special_chars = tuple(list(" #$!?@~^&*()'\"`_:"))

import re

def replaceBracket(sentence):
    rst = re.sub( "(\(((\S|\s)*)(\)_))","",sentence)
    rst = re.sub( "(\(((\S|\s)*)(\)))","",rst)
    return rst

def getHangeulIndex(one_character):
    for i,comp in enumerate(chosung+jungsung+jongsung+ENGS+special_chars):
        if(one_character.upper()==comp):
            return i+4
    return 3
def convertString2HangeulIndex(text):
    result = []
    for character in text:
        result.append(getHangeulIndex(character))
    return result

def isHangeul(one_character):
    return 0xAC00 <= ord(one_character[:1]) <= 0xD7A3

def hangeulExplode(one_hangeul):
    a = one_hangeul[:1]
    if isHangeul(a) != True:
        return False
    b = ord(a) - 0xAC00
    cho = b // (21*28)
    jung = b % (21*28) // 28
    jong = b % 28
    if jong == 0:
        return (chosung[cho], jungsung[jung])
    else:
        return (chosung[cho], jungsung[jung], jongsung[jong])

def hangeulJoin(inputlist):
    result = ""
    cho, jung, jong = 0, 0, 0
    inputlist.insert(0, "")
    while len(inputlist) > 1:
        if inputlist[-1] in jongsung:
            if inputlist[-2] in jungsung:
                jong = jongsung.index(inputlist.pop())
            
            else:
                result += inputlist.pop()
        elif inputlist[-1] in jungsung:
            if inputlist[-2] in chosung:
                jung = jungsung.index(inputlist.pop())
                cho = chosung.index(inputlist.pop())
                result += chr(0xAC00 + ((cho*21)+jung)*28+jong)
                cho, jung, jong = 0, 0, 0
            else:
                result += inputlist.pop()

        else:
            result += inputlist.pop()
    else:
        return result[::-1]

def pureosseugi(inputtext):
    result = ""
    for i in inputtext:
        if isHangeul(i) == True:
            for j in hangeulExplode(i):
                result += j
        else:
            result += i
    
    return result

def moasseugi(inputtext):
    t1 = []
    for i in inputtext:
        t1.append(i)

    return hangeulJoin(t1)

In [11]:
import numpy as np
import os
from glob import glob
import json
from pathlib import Path
import torch
import torchaudio
from torchaudio import transforms as audioT



class DatasetLoader:
    def __init__(self,is_train_dataset:bool,batch_size,top_dataset_folder = "자유대화 음성(일반남녀)"):
        super().__init__()
        section_folder = "Training"
        if(is_train_dataset==False):
            section_folder = "Validation"
        top_folder = os.path.join(top_dataset_folder,section_folder)
        tmp_audios = glob(os.path.join(top_folder,"*\\*.wav"))
        audios = []
        for aud in tmp_audios:
            ffolder = os.path.dirname(aud)
            fname = os.path.basename(aud)
            fn = Path(fname).stem
            trans_path = os.path.join(ffolder,fn+".json")
            if(os.path.exists(trans_path)==True):
                trans = self.__getTranscription(trans_path)
                trans = replaceBracket(trans)
                if(len(trans)>0):
                    audios.append(aud)
        self.__audios = audios
        use_subset = True
        if(use_subset==True):
            audio_len = len(audios)
            self.__audios = self.__audios[:audio_len//10]
        self.__audio_len = len(audios)
        self._batch_size = batch_size
    def __len__(self):
        return self.__audio_len//self._batch_size
    def __getTranscription(self,fpath):
        with open(fpath,"r",encoding="UTF-8") as fp:
            data = json.load(fp)["발화정보"]["stt"]
            data = pureosseugi(data)
            #data = jamo.h2j(data)
            #data = jamo.j2hcj(data)
        return data
    def __getRawAudioData(self,fpath,with_pytorch=True):
        if(with_pytorch==True):
            data,sample_rate = torchaudio.load(fpath)
            return data,sample_rate
        else:
            import librosa
            data,sample_rate = librosa.load(fpath)
            return data,sample_rate

    def __getAudioData(self,fpath,as_raw_data:bool = False,with_pytorch=True):
        data,sample_rate = self.__getRawAudioData(fpath,with_pytorch)
        if(as_raw_data==False):
            if(with_pytorch==True):
                mfcc_transforms = audioT.MFCC(sample_rate=sample_rate,n_mfcc=96)
                data = mfcc_transforms(data)
                data = data.transpose(1,2)
                data = data.squeeze(0)
                length = data.size(0)

            else:
                import librosa
                data= librosa.feature.mfcc(y=data,sr=sample_rate).T
                data = torch.from_numpy(data)
                length = data.size(0)
        return data,length
        
        
    def shuffleDataset(self):
        random.shuffle(self.__audios)
    def __getitem__(self,idx):
        st_idx,ed_idx = idx*self._batch_size,min((idx+1)*self._batch_size,self.__audio_len)
        audios = []
        audio_lengths = []
        transcripts = []
        transcript_length = []
        for i in range(st_idx,ed_idx):
            fpath = self.__audios[i]
            fname = os.path.basename(fpath)
            #print(fname)
            ffolder = os.path.dirname(fpath)
            transcript_fpath = os.path.join(ffolder,Path(fname).stem+".json")
            transcription = self.__getTranscription(transcript_fpath)
            audio_data,data_length = self.__getAudioData(fpath)
            audios.append(audio_data)
            audio_lengths.append(data_length)
            #print("transcript:",transcription)
            transcription = replaceBracket(transcription)
            #print("transcript2:",transcription)
            transcription = convertString2HangeulIndex(transcription)
            transcripts.append(torch.from_numpy(np.array(transcription)))

            transcript_length.append(len(transcription))#temp length
        audios = torch.nn.utils.rnn.pad_sequence(audios,batch_first=True)
        audio_lengths = np.array(audio_lengths)
        transcript_length = np.array(transcript_length)
        #transcripts = np.array(transcripts)
        return audios,audio_lengths,transcripts,transcript_length
        
if(__name__=="__main__"):
    debug_dataset = True
    if(debug_dataset==True):
        dataset_loader = DatasetLoader(is_train_dataset=True,batch_size=2,top_dataset_folder="자유대화 음성(일반남녀)")
        for i, (audios,audio_lengths,transcripts,transcript_length) in enumerate(dataset_loader):
            #print("audios:",len(audios),"transcripts:",len(transcripts))
            sample_audio = audios[0]
            sample_transcripts = transcripts[0]
            print(sample_audio.shape)
            print("  ",sample_transcripts)
            labels = nn.utils.rnn.pad_sequence(transcripts,batch_first=True)
            print("label:",labels.shape)
            if(i>10):
                break


torch.Size([666, 96])
   tensor([ 4, 41,  9, 27,  6, 98, 13, 28, 10, 15, 43, 16, 43, 98,  4, 41, 98, 15,
        43, 22, 36, 98, 18, 27,  6, 22, 32, 15, 15, 43, 98, 16, 43,  6, 23,  7,
        23,  4, 23, 98,  6, 24,  9, 29,  4, 15, 41,  9, 98,  7, 41,  7,  4, 31,
        98, 15, 43,  9, 41, 10, 15, 41,  9, 98,  4, 41, 10, 13, 31,  9, 31,  9,
        31, 98, 11, 23,  5, 37, 14,  7, 24], dtype=torch.int32)
label: torch.Size([2, 81])
torch.Size([698, 96])
   tensor([19, 27,  7, 23,  9, 23,  6, 98, 16, 27,  6, 18, 27,  9, 15, 41,  9, 98,
        20, 23,  4, 31, 98, 15, 27,  7, 43,  9, 31,  6,  4, 23, 98,  4, 23,  4,
        31, 98, 15, 43, 14,  6, 41,  6,  7, 28, 98, 18, 23, 15, 11, 23,  5, 15,
        41,  9, 31, 98, 22, 24, 13, 13, 23,  9, 15, 43, 98,  6, 36,  6, 11, 36,
        13, 43,  4, 28, 98, 11, 36, 13, 27, 16, 43,  7, 27,  9, 23],
       dtype=torch.int32)
label: torch.Size([2, 88])
torch.Size([532, 96])
   tensor([ 4, 41,  9, 43,  4, 31, 98, 15, 35,  4, 23, 22, 23,  6, 41,  6, 

In [2]:
if(__name__=="__main__"):
    traindataset_loader = DataLoader(15)
    from matplotlib import pyplot as plt
    index = 0
    for specs,specs_lengths,transcripts,transcript_lengths in tqdm(traindataset_loader):
        #print(specs.shape,specs_lengths.shape,transcripts.shape,transcript_lengths.shape)
        fig, ax = plt.subplots(nrows=2, sharex=True)
        mfcc= specs[0].T
        print("mfcc:",mfcc.shape)
        print("\t",(mfcc.min(),mfcc.max()))#(np.min(mfcc),np.max(mfcc)))
        img = librosa.display.specshow(librosa.power_to_db(mfcc, ref=np.max),
                                      x_axis='time', y_axis='mel',fmin=100, fmax=8000,
                                      ax=ax[0])
        fig.colorbar(img, ax=[ax[0]])
        ax[0].set(title='Mel spectrogram')
        ax[0].label_outer()
        img = librosa.display.specshow(mfcc, x_axis='time', ax=ax[1])
        fig.colorbar(img, ax=[ax[1]])
        ax[1].set(title='MFCC')
        plt.show()
        if(index>20):
            break
        index+=1


NameError: name 'DataLoader' is not defined

* https://pytorch.org/audio/stable/generated/torchaudio.models.emformer_rnnt_model.html?highlight=rnnt#torchaudio.models.emformer_rnnt_model

In [7]:
net = emformer_rnnt_model(
        input_dim=96,
        encoding_dim=1024,
        num_symbols=200,
        segment_length=16,
        right_context_length=4,
        time_reduction_input_dim=128,
        time_reduction_stride=4,
        transformer_num_heads=8,
        transformer_ffn_dim=1536,
        transformer_num_layers=20,
        transformer_dropout=0.1,
        transformer_activation="gelu",
        transformer_left_context_length=30,
        transformer_max_memory_size=0,
        transformer_weight_init_scale_strategy="depthwise",
        transformer_tanh_on_mem=True,
        symbol_embedding_dim=512,
        num_lstm_layers=3,
        lstm_layer_norm=True,
        lstm_layer_norm_epsilon=1e-3,
        lstm_dropout=0.3,
    )
device = torch.device("cuda:0" if(torch.cuda.is_available()==True) else "cpu")
net = net.to(device)
#net.device

* https://pytorch.org/audio/stable/generated/torchaudio.models.RNNTBeamSearch.html?highlight=beamsearch#torchaudio.models.RNNTBeamSearch

In [9]:
def predict(net,dataset):
    net.eval()
    state = None
    infer = torchaudio.models.RNNTBeamSearch(net,blank=0)
    print("device:",device)
    with torch.no_grad():
        for (features,feat_lens,labels,label_lens) in dataset:
            feat_lens,label_lens = torch.from_numpy(feat_lens),torch.from_numpy(label_lens)

            features = nn.utils.rnn.pad_sequence(features,batch_first=True)
            labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)
            
            features = features.to(device)
            labels = labels.to(device)
            feat_lens = feat_lens.to(device)
            label_lens = label_lens.to(device)
            
            output = infer(features.float(),feat_lens,10)
            tmp_output = output[0]

            print("-"*10)
            print("\toriginal:",labels)
            #with_visible_tag
            ori_string = dataset.token2String(labels.cpu().numpy()[0].tolist())
            pred_string = dataset.token2String(output[0][0])
            print("\tori string: ",ori_string)
            print("\toutput result:",output[0][0])
            print("\topt string: ",pred_string)
            ori_string = dataset.token2String(labels.cpu().numpy()[0].tolist(),with_visible_tag=False)
            pred_string = dataset.token2String(output[0][0],with_visible_tag=False)
            print(f"\topt' string: '{pred_string}'")
            print("\twer:",wer(pred_string,ori_string))
            print("\tcer:",cer(pred_string,ori_string))
            
            del features,feat_lens,labels,label_lens
            torch.cuda.empty_cache()
  

In [12]:
initial_learning_rate = 0.0002
optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.95) #.CosineAnnealingLR(optimizer,T_max=100,eta_min=0.001)

In [10]:
weights = torch.load("./rnnt.pth")
net.load_state_dict(weights)

<All keys matched successfully>

In [18]:
predict(net,testdataset)

device: cuda:0
----------
	original: tensor([[ 0, 10,  7, 29, 10, 17, 18,  7,  6, 29, 22, 10,  7, 20,  7, 29, 25, 17,
         23, 14,  6, 29,  4,  7, 29, 21, 22,  7, 25, 29,  8, 17, 20, 29,  6, 11,
         16, 16,  7, 20, 29, 22, 23, 20, 16, 11, 18, 21, 29,  3, 16,  6, 29,  5,
          3, 20, 20, 17, 22, 21, 29,  3, 16,  6, 29,  4, 20, 23, 11, 21,  7,  6,
         29, 18, 17, 22,  3, 22, 17,  7, 21, 29,  3, 16,  6, 29,  8,  3, 22, 29,
         15, 23, 22, 22, 17, 16, 29, 18, 11,  7,  5,  7, 21, 29, 22, 17, 29,  4,
          7, 29, 14,  3,  6, 14,  7,  6, 29, 17, 23, 22, 29, 11, 16, 29, 22, 10,
         11,  5, 13, 29, 18,  7, 18, 18,  7, 20,  7,  6, 29,  8, 14, 17, 23, 20,
         29,  8,  3, 22, 22,  7, 16,  7,  6, 29, 21,  3, 23,  5,  7]],
       device='cuda:0', dtype=torch.int32)
	ori string:  </s>HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE
	output result: [0, 22, 

----------
	original: tensor([[ 0, 21, 22, 23,  8,  8, 29, 11, 22, 29, 11, 16, 22, 17, 29, 27, 17, 23,
         29, 10, 11, 21, 29,  4,  7, 14, 14, 27, 29,  5, 17, 23, 16, 21,  7, 14,
         14,  7,  6, 29, 10, 11, 15]], device='cuda:0', dtype=torch.int32)
	ori string:  </s>STUFF IT INTO YOU HIS BELLY COUNSELLED HIM
	output result: [0, 22, 10, 7, 20, 7, 29, 5, 17, 23, 14, 6, 29, 4, 7, 29, 14, 11, 22, 22, 14, 7, 29, 3, 20, 22, 29, 11, 16, 29, 22, 10, 11, 21, 29, 14, 3, 21, 22, 29, 3, 16, 6, 29, 8, 11, 16, 3, 14, 29, 20, 17, 23, 16, 6, 29, 17, 8, 29, 8, 7, 16, 5, 11, 16, 9, 0, 0, 10, 7, 29, 7, 3, 22, 21, 29, 3, 16, 6, 29, 21, 14, 7, 7, 18, 21, 29, 24, 7, 20, 27, 29, 21, 22, 7, 3, 6, 11, 14, 27, 29, 20, 7, 18, 14, 11, 7, 6, 29, 22, 10, 7, 29, 16, 7, 25, 29, 13, 11, 16, 9, 0, 0, 11, 29, 21, 17, 15, 7, 22, 10, 11, 16, 9, 29, 5, 14, 17, 21, 7, 29, 22, 17, 29, 18, 3, 16, 11, 5, 29, 17, 16, 29, 10, 11, 21, 29, 17, 18, 18, 17, 16, 7, 16, 22, 31, 21, 29, 8, 3, 5, 7, 29, 25, 10, 7, 16, 29, 22, 

----------
	original: tensor([[ 0,  3,  8, 22,  7, 20, 29,  7,  3, 20, 14, 27, 29, 16, 11,  9, 10, 22,
          8,  3, 14, 14, 29, 22, 10,  7, 29, 27,  7, 14, 14, 17, 25, 29, 14,  3,
         15, 18, 21, 29, 25, 17, 23, 14,  6, 29, 14, 11,  9, 10, 22, 29, 23, 18,
         29, 10,  7, 20,  7, 29,  3, 16,  6, 29, 22, 10,  7, 20,  7, 29, 22, 10,
          7, 29, 21, 19, 23,  3, 14, 11,  6, 29, 19, 23,  3, 20, 22,  7, 20, 29,
         17,  8, 29, 22, 10,  7, 29,  4, 20, 17, 22, 10,  7, 14, 21]],
       device='cuda:0', dtype=torch.int32)
	ori string:  </s>AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS
	output result: [0, 22, 10, 7, 20, 7, 29, 5, 17, 23, 14, 6, 29, 4, 7, 29, 14, 11, 22, 22, 14, 7, 29, 3, 20, 22, 29, 11, 16, 29, 22, 10, 11, 21, 29, 14, 3, 21, 22, 29, 3, 16, 6, 29, 8, 11, 16, 3, 14, 29, 20, 17, 23, 16, 6, 29, 17, 8, 29, 8, 7, 16, 5, 11, 16, 9, 0, 0, 10, 7, 29, 7, 3, 22, 21, 29, 3, 16, 6, 29, 21, 14, 7, 7, 18, 21, 29, 

----------
	original: tensor([[ 0, 10,  7, 14, 14, 17, 29,  4,  7, 20, 22, 11,  7, 29,  3, 16, 27, 29,
          9, 17, 17,  6, 29, 11, 16, 29, 27, 17, 23, 20, 29, 15, 11, 16,  6]],
       device='cuda:0', dtype=torch.int32)
	ori string:  </s>HELLO BERTIE ANY GOOD IN YOUR MIND
	output result: [0, 22, 10, 7, 20, 7, 29, 5, 17, 23, 14, 6, 29, 4, 7, 29, 14, 11, 22, 22, 14, 7, 29, 3, 20, 22, 29, 11, 16, 29, 22, 10, 11, 21, 29, 14, 3, 21, 22, 29, 3, 16, 6, 29, 8, 11, 16, 3, 14, 29, 20, 17, 23, 16, 6, 29, 17, 8, 29, 8, 7, 16, 5, 11, 16, 9, 0, 0, 10, 7, 29, 7, 3, 22, 21, 29, 3, 16, 6, 29, 21, 14, 7, 7, 18, 21, 29, 24, 7, 20, 27, 29, 21, 22, 7, 3, 6, 11, 14, 27, 29, 20, 7, 18, 14, 11, 7, 6, 29, 22, 10, 7, 29, 16, 7, 25, 29, 13, 11, 16, 9, 0, 0, 11, 29, 21, 17, 15, 7, 22, 10, 11, 16, 9, 29, 5, 14, 17, 21, 7, 29, 22, 17, 29, 18, 3, 16, 11, 5, 29, 17, 16, 29, 10, 11, 21, 29, 17, 18, 18, 17, 16, 7, 16, 22, 31, 21, 29, 8, 3, 5, 7, 29, 25, 10, 7, 16, 29, 22, 10, 7, 29, 15, 3, 16, 29, 8, 11, 16, 3, 14

----------
	original: tensor([[ 0, 16, 23, 15,  4,  7, 20, 29, 22,  7, 16, 29,  8, 20,  7, 21, 10, 29,
         16,  7, 14, 14, 27, 29, 11, 21, 29, 25,  3, 11, 22, 11, 16,  9, 29, 17,
         16, 29, 27, 17, 23, 29,  9, 17, 17,  6, 29, 16, 11,  9, 10, 22, 29, 10,
         23, 21,  4,  3, 16,  6]], device='cuda:0', dtype=torch.int32)
	ori string:  </s>NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND
	output result: [0, 22, 10, 7, 20, 7, 29, 5, 17, 23, 14, 6, 29, 4, 7, 29, 14, 11, 22, 22, 14, 7, 29, 3, 20, 22, 29, 11, 16, 29, 22, 10, 11, 21, 29, 14, 3, 21, 22, 29, 3, 16, 6, 29, 8, 11, 16, 3, 14, 29, 20, 17, 23, 16, 6, 29, 17, 8, 29, 8, 7, 16, 5, 11, 16, 9, 0, 0, 10, 7, 29, 7, 3, 22, 21, 29, 3, 16, 6, 29, 21, 14, 7, 7, 18, 21, 29, 24, 7, 20, 27, 29, 21, 22, 7, 3, 6, 11, 14, 27, 29, 20, 7, 18, 14, 11, 7, 6, 29, 22, 10, 7, 29, 16, 7, 25, 29, 13, 11, 16, 9, 0, 0, 11, 29, 21, 17, 15, 7, 22, 10, 11, 16, 9, 29, 5, 14, 17, 21, 7, 29, 22, 17, 29, 18, 3, 16, 11, 5, 29, 17, 16, 29, 10, 

...

----------
	original: tensor([[ 0, 22, 10,  7, 29,  5, 10,  3, 17, 21, 29, 11, 16, 29, 25, 10, 11,  5,
         10, 29, 10, 11, 21, 29,  3, 20,  6, 17, 23, 20, 29,  7, 26, 22, 11, 16,
          9, 23, 11, 21, 10,  7,  6, 29, 11, 22, 21,  7, 14,  8, 29, 25,  3, 21,
         29,  3, 29,  5, 17, 14,  6, 29, 11, 16,  6, 11,  8,  8,  7, 20,  7, 16,
         22, 29, 13, 16, 17, 25, 14,  7,  6,  9,  7, 29, 17,  8, 29, 10, 11, 15,
         21,  7, 14,  8]], device='cuda:0', dtype=torch.int32)
	ori string:  </s>THE CHAOS IN WHICH HIS ARDOUR EXTINGUISHED ITSELF WAS A COLD INDIFFERENT KNOWLEDGE OF HIMSELF
	output result: [0, 22, 10, 7, 20, 7, 29, 5, 17, 23, 14, 6, 29, 4, 7, 29, 14, 11, 22, 22, 14, 7, 29, 3, 20, 22, 29, 11, 16, 29, 22, 10, 11, 21, 29, 14, 3, 21, 22, 29, 3, 16, 6, 29, 8, 11, 16, 3, 14, 29, 20, 17, 23, 16, 6, 29, 17, 8, 29, 8, 7, 16, 5, 11, 16, 9, 0, 0, 10, 7, 29, 7, 3, 22, 21, 29, 3, 16, 6, 29, 21, 14, 7, 7, 18, 21, 29, 24, 7, 20, 27, 29, 21, 22, 7, 3, 6, 11, 14, 27, 29, 20, 7, 18,

----------
	original: tensor([[ 0,  3, 22, 29, 15, 17, 21, 22, 29,  4, 27, 29,  3, 16, 29,  3, 14, 15,
         21, 29,  9, 11, 24,  7, 16, 29, 22, 17, 29,  3, 29,  4,  7,  9,  9,  3,
         20, 29, 25, 10, 17, 21,  7, 29,  4, 14,  7, 21, 21, 11, 16,  9, 29, 10,
          7, 29,  8, 14,  7,  6, 29,  8, 20, 17, 15, 29, 10,  7, 29, 15, 11,  9,
         10, 22, 29, 10, 17, 18,  7, 29, 25,  7,  3, 20, 11, 14, 27, 29, 22, 17,
         29, 25, 11, 16, 29,  8, 17, 20, 29, 10, 11, 15, 21,  7, 14,  8, 29, 21,
         17, 15,  7, 29, 15,  7,  3, 21, 23, 20,  7, 29, 17,  8, 29,  3,  5, 22,
         23,  3, 14, 29,  9, 20,  3,  5,  7]], device='cuda:0',
       dtype=torch.int32)
	ori string:  </s>AT MOST BY AN ALMS GIVEN TO A BEGGAR WHOSE BLESSING HE FLED FROM HE MIGHT HOPE WEARILY TO WIN FOR HIMSELF SOME MEASURE OF ACTUAL GRACE
	output result: [0, 22, 10, 7, 20, 7, 29, 5, 17, 23, 14, 6, 29, 4, 7, 29, 14, 11, 22, 22, 14, 7, 29, 3, 20, 22, 29, 11, 16, 29, 22, 10, 11, 21, 29, 14, 3, 21, 22, 29, 3

----------
	original: tensor([[ 0, 25,  7, 14, 14, 29, 16, 17, 25, 29,  7, 16, 16, 11, 21, 29, 11, 29,
          6,  7,  5, 14,  3, 20,  7, 29, 27, 17, 23, 29, 10,  3, 24,  7, 29,  3,
         29, 10,  7,  3,  6, 29,  3, 16,  6, 29, 21, 17, 29, 10,  3, 21, 29, 15,
         27, 29, 21, 22, 11,  5, 13]], device='cuda:0', dtype=torch.int32)
	ori string:  </s>WELL NOW ENNIS I DECLARE YOU HAVE A HEAD AND SO HAS MY STICK
	output result: [0, 22, 10, 7, 20, 7, 29, 5, 17, 23, 14, 6, 29, 4, 7, 29, 14, 11, 22, 22, 14, 7, 29, 3, 20, 22, 29, 11, 16, 29, 22, 10, 11, 21, 29, 14, 3, 21, 22, 29, 3, 16, 6, 29, 8, 11, 16, 3, 14, 29, 20, 17, 23, 16, 6, 29, 17, 8, 29, 8, 7, 16, 5, 11, 16, 9, 0, 0, 10, 7, 29, 7, 3, 22, 21, 29, 3, 16, 6, 29, 21, 14, 7, 7, 18, 21, 29, 24, 7, 20, 27, 29, 21, 22, 7, 3, 6, 11, 14, 27, 29, 20, 7, 18, 14, 11, 7, 6, 29, 22, 10, 7, 29, 16, 7, 25, 29, 13, 11, 16, 9, 0, 0, 11, 29, 21, 17, 15, 7, 22, 10, 11, 16, 9, 29, 5, 14, 17, 21, 7, 29, 22, 17, 29, 18, 3, 16, 11, 5, 29, 17, 16, 29,

----------
	original: tensor([[ 0, 17, 16, 29, 21,  3, 22, 23, 20,  6,  3, 27, 29, 15, 17, 20, 16, 11,
         16,  9, 21, 29, 25, 10,  7, 16, 29, 22, 10,  7, 29, 21, 17,  6,  3, 14,
         11, 22, 27, 29, 15,  7, 22, 29, 11, 16, 29, 22, 10,  7, 29,  5, 10,  3,
         18,  7, 14, 29, 22, 17, 29, 20,  7,  5, 11, 22,  7, 29, 22, 10,  7, 29,
         14, 11, 22, 22, 14,  7, 29, 17,  8,  8, 11,  5,  7, 29, 10, 11, 21, 29,
         18, 14,  3,  5,  7, 29, 25,  3, 21, 29,  3, 29,  5, 23, 21, 10, 11, 17,
         16,  7,  6, 29, 13, 16,  7,  7, 14, 11, 16,  9, 29,  6,  7, 21, 13, 29,
          3, 22, 29, 22, 10,  7, 29, 20, 11,  9, 10, 22, 29, 17,  8, 29, 22, 10,
          7, 29,  3, 14, 22,  3, 20, 29,  8, 20, 17, 15, 29, 25, 10, 11,  5, 10,
         29, 10,  7, 29, 14,  7,  6, 29, 10, 11, 21, 29, 25, 11, 16,  9, 29, 17,
          8, 29,  4, 17, 27, 21, 29, 22, 10, 20, 17, 23,  9, 10, 29, 22, 10,  7,
         29, 20,  7, 21, 18, 17, 16, 21,  7, 21]], device='cuda:0',
       dtype=torch.

KeyboardInterrupt: 

In [16]:
if(__name__=="__main__"):
    #initial_learning_rate = 0.0001
    #import nemo.collections.asr as nemo_asr

    max_num_epoch = 1
    #optimizer = optim.Adam(net.parameters(),lr=initial_learning_rate)#,rho=0.95,eps=1e-8,weight_decay=0.0)
    criterion = torchaudio.transforms.RNNTLoss(blank=0,reduction='sum',clamp=1.0)#,fused_log_softmax=True)#,#,clamp=1.0)#'sum')
    #criterion = nemo_asr.losses.rnnt.RNNTLoss(num_classes=0)
    traindataset = DatasetLoader(is_train_dataset=True,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    testdataset = DatasetLoader(is_train_dataset=False,batch_size=1,top_dataset_folder="D:\\datasets\\aihub\\남녀자유대화\\자유대화 음성(일반남녀)")
    #torch.save(net.parameters(),"rnnt.pth")
    #predict(net,testdataset)
    
    for epoch in range(max_num_epoch):
        index = 0
        traindataset.shuffleDataset()
        losses = []
        net.train()
        for (features,feat_lens,labels,label_lens) in tqdm((traindataset)):
            feat_lens,label_lens = torch.from_numpy(feat_lens),torch.from_numpy(label_lens)
            
            features = nn.utils.rnn.pad_sequence(features,batch_first=True)#.unsqueeze(1).transpose(2, 3)
            labels = nn.utils.rnn.pad_sequence(labels,batch_first=True)
            
            """
            sorted_lens,indices = torch.sort(feat_lens.view(-1),dim=0,descending=True)
            features = features[indices]
            labels = labels[indices]
            feat_lens = sorted_lens
            label_lens = label_lens[indices]
            """
            
            #features = features.float()
            
            features = features.to(device)
            labels = labels.to(device)
            feat_lens = feat_lens.to(device)
            label_lens = label_lens.to(device)
            
            
            optimizer.zero_grad()

            joint_outputs,src_length,tgt_length,*_ = net(features.float(),feat_lens,labels,label_lens)
            
            outputs = joint_outputs#
            #outputs = nn.functional.log_softmax(joint_outputs,dim=-1)

            #loss = criterion(log_probs=outputs,targets=(labels[...,1:]).int(),input_lengths=(src_length-1).int() ,target_lengths=(label_lens-1).int())
            loss = criterion(outputs,(labels[...,1:]).contiguous().int(),(src_length-1).int() ,(label_lens-1).int())

            
            loss.backward()
            optimizer.step()
            del features,feat_lens,labels,label_lens
            torch.cuda.empty_cache()
            losses.append(loss.item())
            if(index%25==0 and index>0):
                torch.save(net.state_dict(),"rnnt.pth")
            if(index%100==0):
                print(loss.item())
            index+=1
        print(f"ep={epoch}] loss= ",np.mean(losses))
        #scheduler.step()
    torch.save(net.state_dict(),"rnnt.pth")
    predict(net,testdataset)

  0%|                                                                             | 2/46290 [00:04<23:59:53,  1.87s/it]

969.1795043945312


  0%|▏                                                                           | 102/46290 [00:23<2:58:36,  4.31it/s]

149.28338623046875


  0%|▎                                                                           | 201/46290 [00:42<3:15:38,  3.93it/s]

...

 10%|███████▍                                                                   | 4602/46290 [14:38<2:21:48,  4.90it/s]

nan


 10%|███████▌                                                                   | 4629/46290 [14:44<2:12:38,  5.24it/s]

ep=0] loss=  nan
device: cuda:0


RuntimeError: received an empty list of sequences

In [14]:
torch.save(net.state_dict(),"rnnt.pth")

In [15]:
predict(net,testdataset)

device: cuda:0
----------
	original: tensor([[ 0, 10,  7, 29, 10, 17, 18,  7,  6, 29, 22, 10,  7, 20,  7, 29, 25, 17,
         23, 14,  6, 29,  4,  7, 29, 21, 22,  7, 25, 29,  8, 17, 20, 29,  6, 11,
         16, 16,  7, 20, 29, 22, 23, 20, 16, 11, 18, 21, 29,  3, 16,  6, 29,  5,
          3, 20, 20, 17, 22, 21, 29,  3, 16,  6, 29,  4, 20, 23, 11, 21,  7,  6,
         29, 18, 17, 22,  3, 22, 17,  7, 21, 29,  3, 16,  6, 29,  8,  3, 22, 29,
         15, 23, 22, 22, 17, 16, 29, 18, 11,  7,  5,  7, 21, 29, 22, 17, 29,  4,
          7, 29, 14,  3,  6, 14,  7,  6, 29, 17, 23, 22, 29, 11, 16, 29, 22, 10,
         11,  5, 13, 29, 18,  7, 18, 18,  7, 20,  7,  6, 29,  8, 14, 17, 23, 20,
         29,  8,  3, 22, 22,  7, 16,  7,  6, 29, 21,  3, 23,  5,  7]],
       device='cuda:0', dtype=torch.int32)
	ori string:  </s>HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE
	output result: [0, 3, 1

----------
	original: tensor([[ 0, 21, 22, 23,  8,  8, 29, 11, 22, 29, 11, 16, 22, 17, 29, 27, 17, 23,
         29, 10, 11, 21, 29,  4,  7, 14, 14, 27, 29,  5, 17, 23, 16, 21,  7, 14,
         14,  7,  6, 29, 10, 11, 15]], device='cuda:0', dtype=torch.int32)
	ori string:  </s>STUFF IT INTO YOU HIS BELLY COUNSELLED HIM
	output result: [0, 3, 16, 6, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 27, 0, 25, 7, 20, 7, 0, 16, 17, 0, 6, 11, 8, 8, 11, 5, 23, 14, 22, 0, 22, 10, 3, 22, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 5, 17, 23, 16, 22, 20, 27, 0, 3, 16, 6, 29, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 15, 17, 20, 7, 0, 22, 10, 3, 16, 0, 22, 10, 7, 29, 5, 17, 23, 16, 22, 0, 17, 8, 29, 22, 10, 7, 29, 5, 17, 23, 16, 22, 20, 27, 0, 25, 10, 11, 5, 10, 0, 22, 10, 7, 20, 7, 0, 1

----------
	original: tensor([[ 0,  3,  8, 22,  7, 20, 29,  7,  3, 20, 14, 27, 29, 16, 11,  9, 10, 22,
          8,  3, 14, 14, 29, 22, 10,  7, 29, 27,  7, 14, 14, 17, 25, 29, 14,  3,
         15, 18, 21, 29, 25, 17, 23, 14,  6, 29, 14, 11,  9, 10, 22, 29, 23, 18,
         29, 10,  7, 20,  7, 29,  3, 16,  6, 29, 22, 10,  7, 20,  7, 29, 22, 10,
          7, 29, 21, 19, 23,  3, 14, 11,  6, 29, 19, 23,  3, 20, 22,  7, 20, 29,
         17,  8, 29, 22, 10,  7, 29,  4, 20, 17, 22, 10,  7, 14, 21]],
       device='cuda:0', dtype=torch.int32)
	ori string:  </s>AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS
	output result: [0, 3, 16, 6, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 27, 0, 25, 7, 20, 7, 0, 16, 17, 0, 6, 11, 8, 8, 11, 5, 23, 14, 22, 0, 22, 10, 3, 22, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17

----------
	original: tensor([[ 0, 10,  7, 14, 14, 17, 29,  4,  7, 20, 22, 11,  7, 29,  3, 16, 27, 29,
          9, 17, 17,  6, 29, 11, 16, 29, 27, 17, 23, 20, 29, 15, 11, 16,  6]],
       device='cuda:0', dtype=torch.int32)
	ori string:  </s>HELLO BERTIE ANY GOOD IN YOUR MIND
	output result: [0, 3, 16, 6, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 27, 0, 25, 7, 20, 7, 0, 16, 17, 0, 6, 11, 8, 8, 11, 5, 23, 14, 22, 0, 22, 10, 3, 22, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 5, 17, 23, 16, 22, 20, 27, 0, 3, 16, 6, 29, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 15, 17, 20, 7, 0, 22, 10, 3, 16, 0, 22, 10, 7, 29, 5, 17, 23, 16, 22, 0, 17, 8, 29, 22, 10, 7, 29, 5, 17, 23, 16, 22, 20, 27, 0, 25, 10, 11, 5, 10, 0, 22, 10, 7, 20, 7, 0, 11, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10,

----------
	original: tensor([[ 0, 16, 23, 15,  4,  7, 20, 29, 22,  7, 16, 29,  8, 20,  7, 21, 10, 29,
         16,  7, 14, 14, 27, 29, 11, 21, 29, 25,  3, 11, 22, 11, 16,  9, 29, 17,
         16, 29, 27, 17, 23, 29,  9, 17, 17,  6, 29, 16, 11,  9, 10, 22, 29, 10,
         23, 21,  4,  3, 16,  6]], device='cuda:0', dtype=torch.int32)
	ori string:  </s>NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND
	output result: [0, 3, 16, 6, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 27, 0, 25, 7, 20, 7, 0, 16, 17, 0, 6, 11, 8, 8, 11, 5, 23, 14, 22, 0, 22, 10, 3, 22, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 5, 17, 23, 16, 22, 20, 27, 0, 3, 16, 6, 29, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 15, 17, 20, 7, 0, 22, 10, 3, 16, 0, 22, 10, 7, 29, 5, 17, 23, 16, 22, 0, 17, 8,

...

----------
	original: tensor([[ 0,  3, 16,  9, 20, 27, 29, 25, 11, 22, 10, 29, 10, 11, 15, 21,  7, 14,
          8, 29, 10,  7, 29, 22, 20, 11,  7,  6, 29, 22, 17, 29, 10, 11,  6,  7,
         29, 10, 11, 21, 29,  8,  3,  5,  7, 29,  8, 20, 17, 15, 29, 22, 10,  7,
         11, 20, 29,  7, 27,  7, 21, 29,  4, 27, 29,  9,  3, 28, 11, 16,  9, 29,
          6, 17, 25, 16, 29, 21, 11,  6,  7, 25,  3, 27, 21, 29, 11, 16, 22, 17,
         29, 22, 10,  7, 29, 21, 10,  3, 14, 14, 17, 25, 29, 21, 25, 11, 20, 14,
         11, 16,  9, 29, 25,  3, 22,  7, 20, 29, 23, 16,  6,  7, 20, 29, 22, 10,
          7, 29,  4, 20, 11,  6,  9,  7, 29,  4, 23, 22, 29, 10,  7, 29, 21, 22,
         11, 14, 14, 29, 21,  3, 25, 29,  3, 29, 20,  7,  8, 14,  7,  5, 22, 11,
         17, 16, 29, 22, 10,  7, 20,  7, 11, 16, 29, 17,  8, 29, 22, 10,  7, 11,
         20, 29, 22, 17, 18, 29, 10,  7,  3, 24, 27, 29, 21, 11, 14, 13, 29, 10,
          3, 22, 21, 29,  3, 16,  6, 29, 10, 23, 15,  4, 14,  7, 29, 22,  3, 18,
      

----------
	original: tensor([[ 0,  4, 20, 17, 22, 10,  7, 20, 29, 15,  3,  5, 29,  3, 20,  6, 14,  7,
         29,  4, 20, 17, 22, 10,  7, 20, 29, 13,  7, 17,  9, 10]],
       device='cuda:0', dtype=torch.int32)
	ori string:  </s>BROTHER MAC ARDLE BROTHER KEOGH
	output result: [0, 3, 16, 6, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 27, 0, 25, 7, 20, 7, 0, 16, 17, 0, 6, 11, 8, 8, 11, 5, 23, 14, 22, 0, 22, 10, 3, 22, 0, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 18, 20, 7, 21, 7, 16, 5, 7, 0, 17, 8, 29, 22, 10, 7, 29, 5, 17, 23, 16, 22, 20, 27, 0, 3, 16, 6, 29, 22, 10, 7, 20, 7, 0, 25, 3, 21, 0, 16, 17, 0, 15, 17, 20, 7, 0, 22, 10, 3, 16, 0, 22, 10, 7, 29, 5, 17, 23, 16, 22, 0, 17, 8, 29, 22, 10, 7, 29, 5, 17, 23, 16, 22, 20, 27, 0, 25, 10, 11, 5, 10, 0, 22, 10, 7, 20, 7, 0, 11, 21, 0, 16, 17, 0, 17, 16, 7, 0, 22, 10, 3, 22, 0, 22, 

----------
	original: tensor([[ 0, 22, 10,  7, 11, 20, 29, 18, 11,  7, 22, 27, 29, 25, 17, 23, 14,  6,
         29,  4,  7, 29, 14, 11, 13,  7, 29, 22, 10,  7, 11, 20, 29, 16,  3, 15,
          7, 21, 29, 14, 11, 13,  7, 29, 22, 10,  7, 11, 20, 29,  8,  3,  5,  7,
         21, 29, 14, 11, 13,  7, 29, 22, 10,  7, 11, 20, 29,  5, 14, 17, 22, 10,
          7, 21, 29,  3, 16,  6, 29, 11, 22, 29, 25,  3, 21, 29, 11,  6, 14,  7,
         29,  8, 17, 20, 29, 10, 11, 15, 29, 22, 17, 29, 22,  7, 14, 14, 29, 10,
         11, 15, 21,  7, 14,  8, 29, 22, 10,  3, 22, 29, 22, 10,  7, 11, 20, 29,
         10, 23, 15,  4, 14,  7, 29,  3, 16,  6, 29,  5, 17, 16, 22, 20, 11, 22,
          7, 29, 10,  7,  3, 20, 22, 21, 29, 11, 22, 29, 15, 11,  9, 10, 22, 29,
          4,  7, 29, 18,  3, 11,  6, 29,  3, 29,  8,  3, 20, 29, 20, 11,  5, 10,
          7, 20, 29, 22, 20, 11,  4, 23, 22,  7, 29, 17,  8, 29,  6,  7, 24, 17,
         22, 11, 17, 16, 29, 22, 10,  3, 16, 29, 10, 11, 21, 29, 10,  3,  6, 29,
      

----------
	original: tensor([[ 0, 11, 22, 29, 25,  3, 21, 29, 11,  6, 14,  7, 29,  8, 17, 20, 29, 10,
         11, 15, 29, 22, 17, 29, 15, 17, 24,  7, 29, 10, 11, 15, 21,  7, 14,  8,
         29, 22, 17, 29,  4,  7, 29,  9,  7, 16,  7, 20, 17, 23, 21, 29, 22, 17,
         25,  3, 20,  6, 21, 29, 22, 10,  7, 15, 29, 22, 17, 29, 22,  7, 14, 14,
         29, 10, 11, 15, 21,  7, 14,  8, 29, 22, 10,  3, 22, 29, 11,  8, 29, 10,
          7, 29,  7, 24,  7, 20, 29,  5,  3, 15,  7, 29, 22, 17, 29, 22, 10,  7,
         11, 20, 29,  9,  3, 22,  7, 21, 29, 21, 22, 20, 11, 18, 18,  7,  6, 29,
         17,  8, 29, 10, 11, 21, 29, 18, 20, 11,  6,  7, 29,  4,  7,  3, 22,  7,
         16, 29,  3, 16,  6, 29, 11, 16, 29,  4,  7,  9,  9,  3, 20, 31, 21, 29,
         25,  7,  7,  6, 21, 29, 22, 10,  3, 22, 29, 22, 10,  7, 27, 29, 25, 17,
         23, 14,  6, 29,  4,  7, 29,  9,  7, 16,  7, 20, 17, 23, 21, 29, 22, 17,
         25,  3, 20,  6, 21, 29, 10, 11, 15, 29, 14, 17, 24, 11, 16,  9, 29, 10,
      

KeyboardInterrupt: 